In [55]:
import os, subprocess
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType, StructField, StringType
)
from pyspark.sql.functions import col, when, to_date, to_timestamp, coalesce
from pyspark.sql import functions as F
import great_expectations as gx

import numpy as np
import json

JAVA_HOME = subprocess.check_output(
    ["brew", "--prefix", "openjdk@17"], text=True
).strip()

os.environ["JAVA_HOME"] = JAVA_HOME
os.environ["PATH"] = JAVA_HOME + "/bin:" + os.environ["PATH"]


warehouse_path = "file:///Users/user/Documents/Mackenzie Data Enginner/projeto_data_lake/warehouse"

spark = (
    SparkSession.builder
        .appName("EcommercePipeline")
        # carrega o runtime do Iceberg compatível com Spark 4.0 / Scala 2.13
        .config(
            "spark.jars.packages",
            "org.apache.iceberg:iceberg-spark-runtime-4.0_2.13:1.10.0"
        )
        # ativa as extensões do Iceberg
        .config(
            "spark.sql.extensions",
            "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
        )
        # define o catálogo Iceberg "my_catalog"
        .config("spark.sql.catalog.my_catalog", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.my_catalog.type", "hadoop")
        .config("spark.sql.catalog.my_catalog.warehouse", warehouse_path)
        # se quiser que o catálogo padrão seja o my_catalog:
        .config("spark.sql.default.catalog", "my_catalog")
        .getOrCreate()
)



schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer_name", StringType(), True),
    StructField("customer_email", StringType(), True),
    StructField("product_name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("quantity", StringType(), True),
    StructField("price", StringType(), True),
    StructField("discount", StringType(), True),
    StructField("total_price", StringType(), True),
    StructField("payment_method", StringType(), True),
    StructField("order_date", StringType(), True),
    StructField("delivery_date", StringType(), True),
    StructField("shipping_address", StringType(), True),
    StructField("status", StringType(), True),
    StructField("is_gift", StringType(), True),
    StructField("gift_message", StringType(), True)
])

#Ler a camada RAW
df_raw = (
    spark.read
        .schema(schema)
        .option("multiLine", "true")
        .option("mode", "PERMISSIVE")
        .json("ecommerce_data_20251127_143053.json")
)

print("VALIDACAO JSON 52:", df_raw.count())

df_raw.show(5, truncate=False)

# Transformação 1 — Limpeza de valores inválidos"

df_clean = (
    df_raw
    .replace("invalid_value", None)
    .replace("", None)
)

df_casted = (
    df_clean
    .withColumn("quantity", col("quantity").cast("int"))
    .withColumn("price", col("price").cast("double"))
    .withColumn("discount", col("discount").cast("double"))
    .withColumn("total_price", col("total_price").cast("double"))
    .withColumn("is_gift", col("is_gift").cast("boolean"))
    .withColumn("order_date", to_timestamp("order_date", "yyyy-MM-dd HH:mm:ss"))
    .withColumn("delivery_date", to_timestamp("delivery_date", "yyyy-MM-dd HH:mm:ss"))
)

df_casted.show(5, truncate=False)

print("Contagem antes da remoção de nulos essenciais:", df_casted.count())
#Transformação 3 — Remover registros inválidos essenciais
df_valid = df_casted.dropna(subset=["order_id", "customer_name", "price"])
print("Contagem após a remoção de nulos essenciais:", df_valid.count())
#Transformação 4 — Desduplicação por order_id
df_dedup = df_valid.dropDuplicates()
print("Contagem após desduplicação:", df_dedup.count())

df_dedup = (
    df_dedup
    .withColumn("order_year", F.year("order_date"))
    .withColumn("order_month", F.month("order_date"))
)

df_dedup.printSchema()


VALIDACAO JSON 52: 52
+------------------------------------+---------------+---------------------------+------------+--------+--------+------+--------+-----------+--------------+-------------------+-------------------+--------------------------------------------------+---------+-------+--------------------------------------------------------+
|order_id                            |customer_name  |customer_email             |product_name|category|quantity|price |discount|total_price|payment_method|order_date         |delivery_date      |shipping_address                                  |status   |is_gift|gift_message                                            |
+------------------------------------+---------------+---------------------------+------------+--------+--------+------+--------+-----------+--------------+-------------------+-------------------+--------------------------------------------------+---------+-------+--------------------------------------------------------+
|6d7578d5

In [56]:
#Escrever na camada Curated

caminho_parquet = "/Users/user/Documents/Mackenzie Data Enginner/projeto_data_lake/curated"
df_dedup_wr = df_dedup.coalesce(1)


(
    df_dedup_wr.write
    .mode("overwrite")
    .partitionBy("order_year", "order_month") 
    .option("compression", "snappy")
    .parquet(caminho_parquet)
)

print(f"\n--- 1. Escrita em Parquet concluída ---")
print(f"Dados salvos e particionados em: {caminho_parquet}")


--- 1. Escrita em Parquet concluída ---
Dados salvos e particionados em: /Users/user/Documents/Mackenzie Data Enginner/projeto_data_lake/curated


In [57]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS my_catalog.dq_db")

spark.sql("""
CREATE TABLE IF NOT EXISTS my_catalog.dq_db.order (
  order_id         STRING,
  customer_name    STRING,
  customer_email   STRING,
  product_name     STRING,
  category         STRING,
  quantity         INT,
  price            DOUBLE,
  discount         DOUBLE,
  total_price      DOUBLE,
  payment_method   STRING,
  order_date       TIMESTAMP,
  delivery_date    TIMESTAMP,
  shipping_address STRING,
  status           STRING,
  is_gift          BOOLEAN,
  gift_message     STRING,
  order_year       INT,
  order_month      INT
)
USING iceberg
TBLPROPERTIES (
    'format-version' = '2',
    'write.update.mode' = 'merge-on-read',
    'write.delete.mode' = 'merge-on-read',
    'write.merge.mode' = 'merge-on-read'
)
PARTITIONED BY (order_year, order_month)
""")


DataFrame[]

In [58]:
df_dedup.createOrReplaceTempView("stg_order")

In [59]:
spark.sql("""select * from stg_order limit 5""").show(truncate=False)

+------------------------------------+--------------+--------------------------+------------+--------+--------+------+--------+-----------+--------------+-------------------+-------------------+------------------------------------------------+---------+-------+----------------------------------------------+----------+-----------+
|order_id                            |customer_name |customer_email            |product_name|category|quantity|price |discount|total_price|payment_method|order_date         |delivery_date      |shipping_address                                |status   |is_gift|gift_message                                  |order_year|order_month|
+------------------------------------+--------------+--------------------------+------------+--------+--------+------+--------+-----------+--------------+-------------------+-------------------+------------------------------------------------+---------+-------+----------------------------------------------+----------+-----------+
|423

In [60]:
df_dedup.createOrReplaceTempView("stg_order")

spark.sql("""
MERGE INTO my_catalog.dq_db.order AS tgt
USING stg_order AS src
ON tgt.order_id = src.order_id
WHEN MATCHED THEN UPDATE SET
    tgt.price = src.price,
    tgt.quantity = src.quantity,
    tgt.discount = src.discount,
    tgt.total_price = src.total_price,
    tgt.is_gift = src.is_gift,            
    tgt.gift_message = src.gift_message
WHEN NOT MATCHED THEN INSERT *
""")

DataFrame[]

In [61]:
spark.sql("""
SELECT *
FROM my_catalog.dq_db.order.snapshots
""").show(truncate=False, vertical=True)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 committed_at  | 2025-12-03 21:05:06.498                                                                                                                                                                                                                                                                                                                                                                                          

In [63]:
order = spark.sql("""
SELECT *
FROM my_catalog.dq_db.order
""")

In [64]:
order.count()

9635

In [65]:
order.groupBy("order_id").count().orderBy("count", ascending=False).show()

+--------------------+-----+
|            order_id|count|
+--------------------+-----+
|16b5d9b7-3988-45e...|    1|
|99a8417e-b1ac-42f...|    1|
|b0a62a23-f97e-4ab...|    1|
|b3d3ca5b-0cf4-4e8...|    1|
|2d7adf8c-d7fb-4bf...|    1|
|e2a6d3ae-0b9e-43b...|    1|
|f0b7e0e3-12f1-46e...|    1|
|42a9b791-434c-461...|    1|
|96cff7d6-5ddd-410...|    1|
|6a4d0c2b-467a-4bd...|    1|
|a6d2bb6d-ba6a-4cd...|    1|
|9db61cab-6b99-4ca...|    1|
|dd26151e-2651-46e...|    1|
|98ac7557-b684-412...|    1|
|9f5cbb1a-b13a-4e4...|    1|
|1d1a0fae-f57f-484...|    1|
|25174d71-28fd-46d...|    1|
|c87a5429-6b8d-4c0...|    1|
|f7d16b1c-5a72-4ae...|    1|
|4a2c5c1d-6d0e-49a...|    1|
+--------------------+-----+
only showing top 20 rows


In [66]:
order.count()

9635

In [ ]:

# Gera novo dataframe df_dq com colunas auxiliares para validação
df_dq = (
    df_dedup
    .fillna({"discount": 0.0, "quantity": 0})
    .withColumn("quantity", F.col("quantity").cast("int"))
    .withColumn("price", F.col("price").cast("double"))
    .withColumn("total_price", F.col("total_price").cast("double"))
    .withColumn("_total_expected", F.col("price") * F.col("quantity") - F.col("discount"))
    .withColumn("_total_match", F.abs(F.col("total_price") - F.col("_total_expected")) <= 1e-2)
    .withColumn(
        "_delivery_after_order",
        (F.col("order_date").isNull()) |
        (F.col("delivery_date").isNull()) |
        (F.col("delivery_date") >= F.col("order_date"))
    )
    .withColumn(
        "_gift_message_ok",
        (
            ~F.coalesce(F.col("is_gift"), F.lit(False)) &
            (F.col("gift_message").isNull() | (F.trim(F.col("gift_message")) == ""))
        )
        |
        (
            F.coalesce(F.col("is_gift"), F.lit(False)) &
            F.col("gift_message").isNotNull() &
            (F.trim(F.col("gift_message")) != "")
        )
    )
)

# 1) contexto
context = gx.get_context()

# 2) datasource Spark
data_source = context.data_sources.add_spark(name="orders_spark")

# 3) asset baseado em DataFrame
data_asset = data_source.add_dataframe_asset(name="orders_asset")

# 4) batch definition que pega o DF inteiro
batch_def = data_asset.add_batch_definition_whole_dataframe(
    name="orders_batch_def"
)

# 5) conecta seu Spark DataFrame df_dq via batch_parameters
batch = batch_def.get_batch(batch_parameters={"dataframe": df_dq})

# 6) cria o Validator a partir do batch
v = context.get_validator(batch=batch)

# 6) Expectations

v.expect_column_values_to_not_be_null("order_id")
v.expect_column_values_to_be_unique("order_id")

# validações de customer_name e customer_email
v.expect_column_values_to_not_be_null("customer_name", mostly=0.99)
#validação de email com regex
v.expect_column_values_to_match_regex(
    "customer_email",
    r"^[\w\.-]+@[\w\.-]+\.\w+$",
    mostly=0.98,
)

# validações de product_name e category
v.expect_column_values_to_not_be_null("product_name", mostly=0.99)
v.expect_column_values_to_not_be_null("category", mostly=0.95)

#validações de quantity, price, discount, total_price
v.expect_column_values_to_not_be_null("quantity")
v.expect_column_values_to_be_between("quantity", min_value=1, max_value=10000, mostly=0.999)
v.expect_column_values_to_be_of_type("quantity", "IntegerType")

#validações de price, discount, total_price
v.expect_column_values_to_not_be_null("price")
v.expect_column_values_to_be_between("price", min_value=0.0, max_value=None, mostly=0.999)
v.expect_column_values_to_be_between("discount", min_value=0.0, max_value=None, mostly=0.999)
v.expect_column_values_to_be_between("total_price", min_value=0.0, max_value=None, mostly=0.999)

#validações de payment_method e status
v.expect_column_values_to_be_in_set(
    "payment_method",
    ["Credit Card", "Debit Card", "Boleto", "PayPal", "Pix", "Cash", "Bank Transfer"],
    mostly=0.95,
)

#validações de status
v.expect_column_values_to_be_in_set(
    "status",
    ["Pending", "Processing", "Shipped", "Delivered", "Cancelled", "Returned"],
    mostly=0.98,
)

#validações de order_date e delivery_date
v.expect_column_values_to_not_be_null("order_date", mostly=0.99)
v.expect_column_values_to_not_be_null("delivery_date", mostly=0.9)
v.expect_column_values_to_be_of_type("order_date", "TimestampType")
v.expect_column_values_to_be_of_type("delivery_date", "TimestampType")

#validações de shipping_address e is_gift
v.expect_column_values_to_not_be_null("shipping_address", mostly=0.98)
v.expect_column_values_to_be_of_type("is_gift", "BooleanType")

#validações de total_match, delivery_after_order, gift_message_ok
v.expect_column_values_to_be_in_set("_total_match", [True], mostly=0.995)
v.expect_column_values_to_be_in_set("_delivery_after_order", [True], mostly=0.995)
v.expect_column_values_to_be_in_set("_gift_message_ok", [True], mostly=0.999)

# 7) Rodar validação
result = v.validate(result_format="SUMMARY")
print(result)


/Users/user/Library/Python/3.9/lib/python/site-packages/great_expectations/expectations/expectation.py:1604: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(
Calculating Metrics: 100%|██████████| 151/151 [00:16<00:00,  9.21it/s]          

{
  "success": false,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "batch_id": "orders_spark-orders_asset",
          "column": "order_id"
        },
        "meta": {},
        "severity": "critical"
      },
      "result": {
        "element_count": 52,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": [],
        "partial_unexpected_counts": []
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_be_unique",
        "kwargs": {
          "batch_id": "orders_spark-orders_asset",
          "column": "order_id"
        },
        "meta": {},
        "severity": "critical"
      },
      "result": {
       

In [ ]:
import pandas as pd

rows = []

#geração do resumo das validações com dataframe pandas
for r in result["results"]:
    exp = r["expectation_config"]
    kwargs = exp.get("kwargs", {})
    res = r.get("result", {})

    col = kwargs.get("column")  # nem toda expectation tem coluna
    unexpected_percent = res.get("unexpected_percent", 0.0)  # pode não existir para algumas
    element_count = res.get("element_count", 0)
    unexpected_count = res.get("unexpected_count", 0)

    rows.append({
        "column": col,
        "expectation": exp.get("expectation_type"),
        "success": r.get("success", False),
        "mostly": kwargs.get("mostly"),
        "element_count": element_count,
        "unexpected_count": unexpected_count,
        "unexpected_percent": unexpected_percent,
        "success_percent": 100.0 - unexpected_percent if unexpected_percent is not None else None,
    })

df_dq_summary = pd.DataFrame(rows)


df_dq_summary = df_dq_summary.sort_values(
    by=["success", "unexpected_percent"],
    ascending=[True, False]
)

print(df_dq_summary.to_string(index=False))

               column expectation  success  mostly  element_count  unexpected_count  unexpected_percent  success_percent
         _total_match        None    False   0.995             52                33           63.461538        36.538462
_delivery_after_order        None    False   0.995             52                30           57.692308        42.307692
     _gift_message_ok        None    False   0.999             52                30           57.692308        42.307692
             quantity        None    False   0.999             52                 4            7.692308        92.307692
         product_name        None    False   0.990             52                 2            3.846154        96.153846
           order_date        None    False   0.990             52                 2            3.846154        96.153846
     shipping_address        None    False   0.980             52                 2            3.846154        96.153846
        delivery_date        Non